# Part A Group 3

Performing the following reactive behaviors.

- Exploration  -> Explores the colors around
- Love -> Moving Forward (Blue)
- Fear -> Moving back (Yellow))
- Aggression -> Moving forward with speed (Red)
- Curious -> Move around the object (Green)


In [1]:
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable
import operator
#use opencv to covert the depth image to RGB image for displaying purpose
import cv2
import numpy as np

#using realsense to capture the color and depth image
import pyrealsense2 as rs
from RobotClass import Robot
#multi-threading is used to capture the image in real time performance
import threading

#initialize the Robot class
robot = Robot()

class Camera(SingletonConfigurable):
    
    #this changing of this value will be captured by traitlets
    color_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
        
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #set resolution for the depth camera
        self.depth_width = 640
        self.depth_height = 480
        self.depth_fps = 30
        self.configuration.enable_stream(rs.stream.depth, self.depth_width, self.depth_height, rs.format.z16, self.depth_fps)
        # Distance
        self.dist = 500
        #flag to control the thread
        self.thread_runnning_flag = False
        
        #start the RGBD sensor
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        #start capture the first color image
        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.color_value = image

        #start capture the first depth image
        depth_frame = frames.get_depth_frame()           
        depth_image = np.asanyarray(depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        self.depth_value = depth_colormap 
        

    def _capture_frames(self):
        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            frames = self.pipeline.wait_for_frames() #receive data from RGBD sensor
            
            color_frame = frames.get_color_frame() #get the color image
            image = np.asanyarray(color_frame.get_data()) #convert color image to numpy array
            self.color_value = image #assign the numpy array image to the color_value variable 

            depth_frame = frames.get_depth_frame() #get the depth image           
            depth_image = np.asanyarray(depth_frame.get_data()) #convert depth data to numpy array
                
            #we only consider the central area of the vision sensor
            depth_image[:190,:]=0
            depth_image[290:,:]=0
            depth_image[:,:160]=0
            depth_image[:,480:]=0


            
            #For object avoidance, we don't consider the distance that are lower than 100mm or bigger than 1000mm
            depth_image[depth_image<100]=0
            depth_image[depth_image>1000]=0
            
            #If all of the values in the depth image is 0, the depth[depth!=0] command will fail
            #we set a specific value here to prevent this failure
            depth_image[0,0]=2000
            
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            self.distance = depth_image[depth_image!=0].min()
            if (depth_image[depth_image!=0].min()<500):
                cv2.putText(self.color_value, 'Avoiding collision!!!', (320,240), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                self.warning_flag=1
                print('Avoiding collision!!!')
                for j in range(0,9):
                    robot.backward(0.3)
                for j in range(0,9):
                    robot.right(0.5)
                
            else:
                self.warning_flag=0
            self.depth_value = depth_colormap   
    
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread       

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])



In [2]:
import time
import ipywidgets.widgets as widgets
from IPython.display import display, HTML
import sys
from RobotClass import Robot
#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data




#initialize the Robot class
robot = Robot()


#create widgets for the displaying of the image
display_color = widgets.Image(format='jpeg', width='45%') #determine the width of the color image
display_depth = widgets.Image(format='jpeg', width='45%')  #determine the width of the depth image
layout=widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth],layout=layout) #horizontal 
display(sidebyside) #display the widget







def operation(result_colour,res,robot):
    if result_colour[res]!=0:
            # Checks if max color is red
            if res=='r':
                # Performs action for detecting red color
                print('back forth')
                for i in range(0,2):
                    for j in range(0,9000):
                        robot.set_motors(-(0.5/3),-0.5,-(0.5)/3,-0.5)
                    for j in range(0,3000):
                        robot.set_motors(0.5,0.5,0.5,0.5)
            # Checks if max color is blue
            elif res =='b':
                # Performs action for detecting blue color
                print('blue forward')
                for i in range(0,27):
                    robot.set_motors(0.3,0.3,0.3,0.3)
                for i in range(0,18):
                    robot.set_motors(0,0,0,0)
            # Checks if max color is yellow
            elif res=='y':
                # Performs action for detecting yellow color
                print('zigzag')
                for i in range(0,2):
                    for j in range(0,1998):
                        robot.set_motors(-0.5,-(0.5/3),-0.5,-(0.5/3))
                    for j in range(0,1998):
                        robot.set_motors(-(0.5/3),-0.5,-(0.5)/3,-0.5)
            # Checks if max color is green
            elif res=='g':
                # Performs action for detecting green color
                print('Rotate')
                for j in range(0,18000):
                    robot.set_motors(-1, 1,-1,1)
    else:
        # Makes the robot go forward.
        print('Move forward!!!!!')
        for j in range(0,18):
            robot.set_motors(0.3,0.3,0.3,0.3)

#callback function, invoked when traitlets detects the changing of the color image
def process(change):
    camera.color_value[:140,:]=0
    if camera.distance<=800:
        print('Detected!!')
        image = change['new'] #retrieve data from the input dict2000/9
        # To show the bottom of the image.

        
        frame = cv2.GaussianBlur(camera.color_value, (5, 5), 0)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        #         hsv = cv2.cvtColor(camera.color_value, cv2.COLOR_BGR2HSV)
        result_colour = {}
        result_col = {}
        # Lower and upper threshold for red color [2]
        lower_red = np.array ([160,50,50])
        upper_red = np.array([180,255,255])
        kernel = np.ones((5,5),'uint8')
 
        mask0 = cv2.inRange(hsv, lower_red, upper_red)
        # Mask for filtering red
        mask = cv2.erode(mask0, None, iterations=2)
        red_mask = cv2.GaussianBlur(mask, (3, 3), 0)
 
        result_red = cv2.bitwise_and(camera.color_value, camera.color_value, mask = red_mask)
        # Store the max value for red.
        result_colour['r'] = np.count_nonzero(result_red)
        result_col['r'] = result_red

        # Define the lower and upper threshold for yellow [3]
        lower_yellow = np.array([22, 93, 0], dtype="uint8")
        upper_yellow = np.array([45, 255, 255], dtype="uint8")
        
        # Mask for filtering yellow
        mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
        result_yellow = cv2.bitwise_and(camera.color_value, camera.color_value, mask = mask_yellow)
        
        # Store the max value for yellow
        result_colour['y'] = np.count_nonzero(result_yellow)
        result_col['y'] = result_yellow

        
        # Define the lower and upper threshold for blue[5]
        lower_blue = np.array([100,150,0],np.uint8)# [180,255,255]
        upper_blue = np.array([140,255,255],np.uint8)
        
        # Mask for filtering blue
        mask2 = cv2.inRange(hsv, lower_blue, upper_blue)
        result_blue = cv2.bitwise_and(camera.color_value, camera.color_value, mask = mask2)
        
        # Store the max value for blue
        result_colour['b'] = np.count_nonzero(result_blue)
        result_col['b'] = result_blue
        

        # Define the lower and upper threshold for green.[4]
        lower_green = np.array ([40, 40,40])
        upper_green = np.array([70, 255,255])
        kernel = np.ones((5,5),'uint8')
        
        # Mask for filtering green.
        mask0 = cv2.inRange(hsv, lower_green, upper_green)

  

        result_green = cv2.bitwise_and(camera.color_value, camera.color_value, mask = mask0)
        # Store the max value for green.
        result_colour['g'] = np.count_nonzero(result_green)
        result_col['g'] = result_green
        
        print('Dict:::',result_colour)
        res = max(result_colour.items(), key=operator.itemgetter(1))[0]
        # Calculates the max out of red , green, yellow and blue.
        print('check:::',[res,result_colour[res]])
        # Creates a thread for running the operations.
        t1 = threading.Thread(target=operation,args=(result_colour,res,robot))
        t1.start()
        t1.join()
        display_color.value = bgr8_to_jpeg(cv2.resize(camera.color_value,(320,240)))
        display_depth.value = bgr8_to_jpeg(cv2.resize(result_col[res],(320,240)))
        cv2.putText(camera.color_value, res, (320,240), cv2.FONT_HERSHEY_SIMPLEX, 10, (0, 0, 255), 2, cv2.LINE_AA)

    else:
        
        robot.set_motors(0.3,0.3,0.3,0.3)
        time.sleep(0.5)
        display_color.value = bgr8_to_jpeg(cv2.resize(camera.color_value,(320,240)))

#the camera.observe function will monitor the color_value variable. If this value changes, the processing function will be excuted.
camera.observe(process, names='color_value')

In [4]:
camera.unobserve_all()
camera.stop()
robot.stop()

Dict::: {'r': 12178, 'y': 9093, 'b': 31007, 'g': 25704}
check::: ['b', 31007]
blue forward
Detected!!
Dict::: {'r': 14977, 'y': 7288, 'b': 29198, 'g': 23202}
check::: ['b', 29198]
blue forward
Detected!!
Dict::: {'r': 19875, 'y': 2898, 'b': 28706, 'g': 22197}
check::: ['b', 28706]
blue forward


# References

- [1]NVIDIA’s jetbot project website: https://github.com/NVIDIA-AI-IOT/jetbot


- [2]https://stackoverflow.com/questions/38877102/how-to-detect-red-color-in-opencv-python


- [3]https://stackoverflow.com/questions/57262974/tracking-yellow-color-object-with-opencv-python


- [4]https://stackoverflow.com/questions/47483951/how-to-define-a-threshold-value-to-detect-only-green-colour-objects-in-an-image


- [5]https://stackoverflow.com/questions/17878254/opencv-python-cant-detect-blue-objects